In [1]:
import pandas as pd
train = pd.read_csv('../input/tps-apr-22-fe-v2/train_tps_apr_22_fev2.csv')
test = pd.read_csv('../input/tps-apr-22-fe-v2/test_tps_apr_22_fev2.csv')

In [2]:
groups = train['subject']
labels = train['state']
train.drop(['sequence', 'subject', 'state'], axis=1, inplace=True)
train.head()

,Unnamed: 0,sensor_00_freq_0,sensor_00_freq_1,sensor_00_freq_2,sensor_00_freq_3,sensor_00_freq_4,sensor_00_freq_5,sensor_00_freq_6,sensor_00_freq_7,sensor_00_freq_8,...,sensor_12_freq_22,sensor_12_freq_23,sensor_12_freq_24,sensor_12_freq_25,sensor_12_freq_26,sensor_12_freq_27,sensor_12_freq_28,sensor_12_freq_29,sensor_12_freq_30,n_sequence_percentile
0,0,2.504637,2.595997,2.948970,2.906888,2.510662,3.610021,6.967675,5.406874,5.151776,...,25.495893,45.878652,32.379605,32.416223,41.594143,34.651699,39.046842,31.815038,27.425405,98.065476
1,1,2.015456,2.296530,1.811023,2.609596,5.933611,5.177809,3.282827,6.652460,7.566777,...,44.968867,16.436525,22.797195,37.939752,20.025530,20.446737,11.834383,0.510406,15.833760,98.065476
2,2,9.068006,13.507249,26.911191,45.104476,66.001563,87.674315,101.652947,109.029241,97.340656,...,129.840977,100.342807,113.971795,138.361735,59.606223,100.369674,19.717507,61.806974,31.247656,98.065476
3,3,0.209428,0.663546,0.330469,0.387669,0.579110,3.711075,3.386412,8.453457,4.531713,...,66.573015,23.868128,36.819744,2.430822,18.272264,9.867578,24.285402,26.592332,23.478687,98.065476
4,4,0.863988,0.508295,1.358652,0.830933,3.506715,2.506207,2.704869,5.660748,6.327061,...,56.084252,37.664327,58.851105,49.871422,31.984101,47.527202,20.673605,38.302775,42.216539,98.065476


In [3]:
params = {'max_depth': 8, 
          'n_estimators': 3000, 
          'learning_rate': 0.08530569310433453, 
          'subsample': 0.2, 
          'colsample_bytree': 0.9, 
          'colsample_bylevel': 0.30000000000000004, 
          'min_child_weight': 0.467428476013027, 
          'reg_lambda': 0.2279498019218528, 
          'reg_alpha': 171.24877820234045, 
          'gamma': 1.2435329218902063}

In [4]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb

kf = GroupKFold(n_splits=5)
scores = []

for fold_idx, (train_index, test_index) in enumerate(kf.split(train, labels, groups)):
    X_train_group = train.iloc[train_index] 
    X_valid_group = train.iloc[test_index]
    y_train_group = labels.iloc[train_index] 
    y_valid_group = labels.iloc[test_index]
    
    model = xgb.XGBClassifier(**params, tree_method='gpu_hist', random_state=127)
    model.fit(X_train_group, y_train_group, eval_set = [(X_valid_group, y_valid_group)], 
                early_stopping_rounds=300, verbose=False)
    fold_score = roc_auc_score(y_valid_group, model.predict_proba(X_valid_group)[:, 1])
    scores.append(fold_score)
    
    print(f'kfold: {fold_idx + 1} , Fold score: {fold_score}')
    
print(f'cv score: {sum(scores) / 5}')

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
kfold: 1 , Fold score: 0.8620598556313128


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
kfold: 2 , Fold score: 0.8712482571420097


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
kfold: 3 , Fold score: 0.883911169299923


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
kfold: 4 , Fold score: 0.8654722237930633


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:47] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
kfold: 5 , Fold score: 0.8750796927034511
cv score: 0.8715542397139521


In [5]:
submission = test[['sequence']]
predictions = pd.DataFrame(
    [model.predict_proba(test.drop(['sequence', 'subject'], axis=1))[:, 1]]).T

submission['state'] = predictions
submission.to_csv('tps_apr_22_xgboost_v0.csv', index=False)
submission.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,sequence,state
0,25968,0.727329
1,26059,0.628526
2,26176,0.715106
3,26249,0.714696
4,26258,0.766382
